In [1]:
import json
import numpy as np
import pandas as pd
import time
import pickle
import datetime
import os
import sys
import requests
from tqdm import tqdm
from FTX_SDK.FTX_SDK import *

api_key = ""
api_secret = ""
subaccount_name = ""
Client = FtxClient(api_key=api_key,api_secret=api_secret,subaccount_name=subaccount_name)

In [27]:
def get_symbol_list():
    
    # existed - black list + delisted 
    
    root_path = os.getcwd() 
    data_path = root_path + '\Data'
    Client = FtxClient(api_key = '', api_secret = '',subaccount_name= '')
    df = pd.DataFrame(Client.get_futures())
    symbol_list = list(df[df['type'] == 'perpetual']['underlying'])
    os.chdir(data_path)
    unenabled_symbols = list(pd.read_csv('black_list.txt',header=None)[0])
    delisted_list = list(pd.read_csv('delisted_list.txt',header=None)[0])
    os.chdir(root_path)
    enabled_symbols = []
    for symbol in symbol_list:
        if(symbol not in unenabled_symbols):
            enabled_symbols.append(symbol)
    symbol_list = enabled_symbols+delisted_list
    return list(set(symbol_list))

In [2]:
def get_OHLCV_df(symbol,file_path):
    file = f'{file_path}/{symbol}_PERP.csv'
    df = pd.read_csv(file, sep = ",", encoding = "utf-8", engine = "c")
    df['startTime'] = pd.to_datetime(df['startTime'])
    df = df.rename(columns={'startTime':'datetime'}).set_index('datetime')
    #df['asset'] = symbol+"-PERP"
    df = df.drop(columns=['time'])
    return df[:-1]

In [3]:
def get_FR_df(symbol,FR_path):
    df = pd.read_csv(f'{FR_path}/{symbol}_FR.csv',sep = ",", encoding = "utf-8", engine = "c")
    df['time'] = pd.to_datetime(df['time'])
    df = df.rename(columns = {'time':'datetime',"rate":'FR'})
    df['datetime'] = df['datetime']-pd.Timedelta('1H')
    df = df.set_index('datetime')
    df = df[['FR']]
    return df

In [4]:
def get_LR_df(symbol,LR_path):
    try:
        df = pd.read_csv(f'{LR_path}/{symbol}_Lending.csv',sep = ",", encoding = "utf-8", engine = "c")
        df['time'] = pd.to_datetime(df['time'])
        df = df.rename(columns = {'time':'datetime',"rate":'LR'})
        df['datetime'] = df['datetime']-pd.Timedelta('1H')
        df = df.set_index('datetime')
        df = df[['LR']]
    except:
        df = pd.DataFrame([],columns=['LR'])
    return df

In [34]:
def get_data(symbol,frequency,data_path = r'C:\Users\Chiuan1007\Crypto\Qlib-github\qlib-crypto-examples\crypto_data'):
    
    df_OHLCV = get_OHLCV_df(symbol = symbol,file_path = r'Y:\數字貨幣_陳冠全\Data\FTX Data\1hour')
    df_FR = get_FR_df(symbol = symbol,FR_path = r'Y:\數字貨幣_陳冠全\Data\FTX Data\Funding Rate')
    df_LR = get_LR_df(symbol = symbol,LR_path = r'Y:\數字貨幣_陳冠全\Data\FTX Data\Lending Rate')
    if(len(df_LR) == 0):
        data = pd.concat([df_OHLCV,df_FR],axis=1).dropna(subset=['open'])
        data = data.resample(frequency).agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum',
                                          'FR':"mean"})
        data['LR'] = np.nan
    else:
        data = pd.concat([df_OHLCV,df_FR,df_LR],axis=1).dropna(subset=['open'])
        data = data.resample(frequency).agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum',
                                          'FR':"mean",'LR':'mean'})

    data['asset'] = symbol
    data['factor'] = 1
    data['vwap'] = (data['close']+ data['high']+ data['low'])/3
    data['change'] = (data['close'].shift(1) - data['close'])/data['close']
    data = data[:-1].reset_index().rename(columns = {'index':'datetime'})
    
    data.to_csv('{}/{}.csv'.format(data_path,symbol),index=False)
    
    return data

In [35]:
symbol = 'FTT'
frequency = '24H'
data_path = r'C:\Users\Chiuan1007\.qlib\csv_data\crypto_data'
data = get_data(symbol,frequency,data_path)
data

,datetime,open,high,low,close,volume,FR,LR,asset,factor,vwap,change
0,2021-01-03 00:00:00+00:00,6.41075,7.0530,6.3760,6.9900,4.683671e+06,0.000129,NaN,FTT,1,6.806333,NaN
1,2021-01-04 00:00:00+00:00,6.99000,7.7090,6.3725,7.5720,8.432562e+06,0.000132,NaN,FTT,1,7.217833,-0.076862
2,2021-01-05 00:00:00+00:00,7.57200,8.7820,7.0715,8.7005,7.081985e+06,0.000087,NaN,FTT,1,8.184667,-0.129705
3,2021-01-06 00:00:00+00:00,8.70050,9.4365,8.3935,9.4255,9.381740e+06,0.000091,NaN,FTT,1,9.085167,-0.076919
4,2021-01-07 00:00:00+00:00,9.42550,10.1210,9.1650,9.6000,1.182616e+07,0.000098,NaN,FTT,1,9.628667,-0.018177
...,...,...,...,...,...,...,...,...,...,...,...,...
565,2022-07-22 00:00:00+00:00,29.49250,30.6900,28.6245,28.9120,4.431262e+07,0.000008,NaN,FTT,1,29.408833,0.020078
566,2022-07-23 00:00:00+00:00,28.91200,29.7510,27.6775,28.5150,2.678566e+07,0.000003,NaN,FTT,1,28.647833,0.013922
567,2022-07-24 00:00:00+00:00,28.51500,29.7820,28.3450,29.1195,2.408590e+07,-0.000004,NaN,FTT,1,29.082167,-0.020759
568,2022-07-25 00:00:00+00:00,29.11950,29.2970,26.9175,27.0115,2.815795e+07,0.000002,NaN,FTT,1,27.742000,0.078041


In [36]:
get_symbol_list()

['LEO',
 'AR',
 'ROOK',
 'FIL',
 'KSM',
 'DMG',
 'ALCX',
 'GLMR',
 'OKB',
 'MINA',
 'ENS',
 'EGLD',
 'HOLY',
 'LOOKS',
 'CVC',
 'DENT',
 'SOS',
 'ALGO',
 'APE',
 'DYDX',
 'C98',
 'MTL',
 'CHR',
 'POLIS',
 'ROSE',
 'ATOM',
 'SLP',
 'BTC',
 'NEO',
 'SECO',
 'AXS',
 'BIT',
 'TULIP',
 'BAT',
 'OP',
 'BAO',
 'BSV',
 'FIDA',
 'MER',
 'DOGE',
 'MCB',
 'CHZ',
 'PERP',
 'IMX',
 'BAND',
 'YFII',
 'BAL',
 'SRN',
 'SNX',
 'ICX',
 'AMPL',
 'REEF',
 'RNDR',
 'ADA',
 'LINA',
 'MNGO',
 'AUDIO',
 'CRO',
 'EOS',
 'CELO',
 'HNT',
 'CVX',
 'ZIL',
 'XTZ',
 'STMX',
 'FLOW',
 'LDO',
 'FTT',
 'BTT',
 'NEAR',
 'SHIB',
 'CREAM',
 'OXY',
 'ZEC',
 'BNT',
 'PROM',
 'ATLAS',
 'VET',
 'XEM',
 'WAVES',
 'KNC',
 'AAVE',
 'FXS',
 'PUNDIX',
 'MATIC',
 'XMR',
 'DODO',
 'STX',
 'DAWN',
 'ENJ',
 'AGLD',
 'ICP',
 'OMG',
 'HOT',
 'BOBA',
 'QTUM',
 'DOT',
 'FTM',
 'RAMP',
 'SPELL',
 'PEOPLE',
 'RUNE',
 'ONE',
 'ALPHA',
 'SXP',
 'SUSHI',
 'YFI',
 'HBAR',
 'BNB',
 'COMP',
 'ANC',
 'ETH',
 'GRT',
 'LRC',
 'REN',
 'RSR',
 'THETA'

In [37]:
frequency = '24H'
data_path = r'C:\Users\Chiuan1007\.qlib\csv_data\crypto_data'

for symbol in tqdm(get_symbol_list()):
    data = get_data(symbol,frequency,data_path)

100%|████████████████████████████████████████████████████████████████████████████████| 173/173 [00:15<00:00, 11.38it/s]


In [38]:
data

,datetime,open,high,low,close,volume,FR,LR,asset,factor,vwap,change
0,2020-09-22 00:00:00+00:00,4.3475,7.148,4.075,5.159,2.260496e+07,-0.000103,NaN,AVAX,1,5.460667,NaN
1,2020-09-23 00:00:00+00:00,5.1590,5.210,3.385,3.518,2.274311e+07,-0.000228,NaN,AVAX,1,4.037667,0.466458
2,2020-09-24 00:00:00+00:00,3.5180,4.854,3.422,4.606,1.330667e+07,-0.000118,NaN,AVAX,1,4.294000,-0.236214
3,2020-09-25 00:00:00+00:00,4.6060,5.020,4.179,4.692,1.154678e+07,-0.000216,NaN,AVAX,1,4.630333,-0.018329
4,2020-09-26 00:00:00+00:00,4.6920,4.798,4.347,4.520,4.545360e+06,-0.000131,NaN,AVAX,1,4.555000,0.038053
...,...,...,...,...,...,...,...,...,...,...,...,...
668,2022-07-22 00:00:00+00:00,25.0000,25.554,23.582,23.622,1.227478e+08,-0.000009,0.000003,AVAX,1,24.252667,0.058335
669,2022-07-23 00:00:00+00:00,23.6220,24.647,22.821,23.587,9.741428e+07,-0.000010,0.000003,AVAX,1,23.685000,0.001484
670,2022-07-24 00:00:00+00:00,23.5870,24.509,23.232,23.788,8.081291e+07,-0.000012,0.000003,AVAX,1,23.843000,-0.008450
671,2022-07-25 00:00:00+00:00,23.7880,23.845,20.882,20.920,1.320500e+08,-0.000006,0.000003,AVAX,1,21.882333,0.137094


In [ ]:
# python scripts/dump_bin.py dump_all --csv_path ~/.qlib/csv_data/crypto_data --qlib_dir ~/.qlib/qlib_data/crypto_data --symbol_field_name asset --date_field_name datetime --include_fields open,high,low,close,volume,FR,BR,factor,vwap,change